In [ ]:
import os
import sys
import json
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
  sys.path.append(module_path)

%env OPENAI_API_KEY = "*"
from PIL import Image
from IPython.core.display import HTML
from functools import partial

from engine.utils import ProgramGenerator, ProgramInterpreter
from prompts.gqa_agent import *
from engine.step_interpreters import parse_step

In [ ]:
prompter = partial(create_prompt_stepname_selective)
generator = ProgramGenerator(prompter=prompter)
interpreter = ProgramInterpreter(dataset='gqa')

In [ ]:
def run_case(image_path, question):
    image = Image.open(image_path)
    image.thumbnail((640,640),Image.Resampling.LANCZOS)
    state = dict(
        IMAGE=image.convert('RGB')
    )
    caption, _, _ = interpreter.execute('X=CAP(image=IMAGE0)',state,inspect=True)
    state['context'] = f'multimodal input: IMAGE ({image.size[0]}wx{image.size[1]}h)\n\nCAPTION_IMAGE="{caption}"\n\nQ="{question}"\n\n<End of Context>\n\nThought(Analyze the question: '
    shots = 0
    success = False
    while shots < 15:
        response, _ = generator.generate(dict(input=state['context']))
        state['context'] += response + '))'
        try:
            statement = response.split('Implement(')[1] + ')'
            result, prog_state, html_str = interpreter.execute(statement,state,inspect=True)
        except:
            return '', state['context']
        step_name = parse_state[statement]['step_name']
        if 'CROP' not in step_name:
            state['context'] += '\n<result>\n' + str(result) + '\n</result>\nThought(Analyze the result: '
        elif 'RESULT' in step_name:
            return result, state['context']
        else:
            output_var = step_name = parse_state[statement]['output_var']
            caption, _, _ = interpreter.execute(f'X=CAP(image={output_var})',state,inspect=True)
            state['context'] += '\n<result>\n' + caption + '\n</result>\nThought(Analyze the result: '
        shots += 1